# Visualização de Dados Covid 19

## Alunos
Alexandre Brito Gomes - 11857323

Diógenes Silva Pedro - 11883476

Gabriel Freitas Ximenes Vasconcelos - 11819084

Neste visualização, analisaremos o conjunto de dados "Coronavirus - Brazil", disponível no Kaggle em https://www.kaggle.com/datasets/unanimad/corona-virus-brazil?select=brazil_covid19_cities.csv . Nele, é possível obter informações sobre o número de casos de Coronavirus no Brasil de maneira temporal: apresentando o número de casos, de maneira cumulativa, desde 30 de janeiro de 2020, quando o primeiro caso suspeito foi encontrado no país. 

Esse conjunto de dados é composto por 6 arquivos csv, contendo dados como latitude e longitude das cidades, data da confirmação de primeiro caso, número de casos confirmados, número de mortes, recuperados, suspeitos, entre diversos outros.

Com isso, é possível fazer análises como verificação de quais localidades tiveram maiores números de casos em pouco tempo, como foi a evolução dos casos ao longo do tempo 

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import json
from urllib.request import urlopen
import math

# Importação dos datasets base

In [ ]:
data_path = 'covid-data/'

In [ ]:
df_population = pd.read_csv(data_path + 'brazil_population_2019.csv')
df_population

In [ ]:
df_cities = pd.read_csv(data_path + "brazil_covid19_cities.csv")
df_cities

In [ ]:
df_coordinates = pd.read_csv(data_path + "brazil_cities_coordinates.csv")
df_coordinates

# Criação do df_city

In [ ]:
df_city = df_cities.merge(df_population[['city', 'population', 'region']], how='inner', left_on = 'name', right_on='city')
df_city = df_city.merge(df_coordinates[['city_name', 'city_code', 'lat', 'long']], how='inner', left_on='name', right_on='city_name')
df_city = df_city.drop(["code", "name", "city_name"], axis='columns')
df_city.head()

In [ ]:
df_city.sort_values(['city', 'date'], inplace=True)
df_city.reset_index(inplace=True, drop=True)

## Resolver o problema da acumulação

As colunas `death` e `mortes` são colunas que idealmente deveriam ser acumuladas. No entanto, ao analisar o dataset podemos ver que há casos em que valores diminuem.
O dataset possui 2770 municípios e apenas 266 não possuem esse problema, por isso, devemos dar a devida atenção a esse problema nos dados.

Para amenizar isso, criamos a função `fix_accumulation` que funciona da seguinte maneira:

1. Defino como `start` a posição cuja a posição seguinte é um valor menor
2. Percorro o array até achar uma posição cujo valor é igual ou superior ao valor do `start`. Essa posição é a `end`
3. Vejo a quantidade de elementos entre `start` e `end` e faço a seguinte equação que representa o coeficiente angular da reta:
   $$
   \Delta y = \frac{\text{arr[end]} - \text{arr[start]}}{\text{end}-\text{start}}
   $$
4. Incremento $\Delta y$ a partir de cada valor, começando de `start` até chegar ao `end`
5. Caso um `start` comece e o array termine antes de chegar ao `end`, os valores seguintes ao `start` são substituídos por ele

In [ ]:
def fix_accumulation(original_arr):
    arr = original_arr.copy()
    prev = arr[0]
    start = 0
    end = 0
    interval = False
    for i, el in enumerate(arr):
        if el < prev and interval == False:
            start = i-1
            interval = True
            
        if el >= arr[start] and interval == True:
            end = i
            interval=False
            delta = (arr[end]-arr[start])/(end-start)
            for j in range(start+1, end):
                arr[j] = int(math.floor(arr[j-1] + delta))
                
        prev = arr[i]
        
    if interval == True:
        for i in range(start+1, len(arr)):
            arr[i] = arr[start]
            
    return arr

In [ ]:
def solucionar_coluna(coluna):
    # Cidades que possuem esse problema
    problemas = list(df_city.loc[df_city[coluna + ' per day'] < 0].drop_duplicates(subset=['city'], inplace=False)["city"])
    print(f"Cidades com o problema na coluna {coluna}: {len(problemas)}")
    
    # lista de índices das linhas de cidades com problemas
    index_list = df_city.loc[df_city['city'].isin(problemas)].index
    
    # Crio uma lista de listas em que cada lista interna são as linhas de uma cidade
    lists_of_cases = [g.values.tolist() for _, g in df_city.loc[df_city['city'].isin(problemas)].groupby(df_city['city'])[coluna]]
    
    # Aplico o fix_accumulation para cada cidade
    fixed_cases = [fix_accumulation(city) for city in lists_of_cases]
    
    # Corrigo a coluna cases para cada cidade
    index_start = 0
    for fixed_city in fixed_cases:
        city_indexes = index_list[index_start: index_start + len(fixed_city)]
        index_start += len(fixed_city)
        df_city.loc[city_indexes, coluna]=fixed_city
        
    # Quantidade de cidades com acumulação incorreta
    problemas = df_city.loc[df_city[coluna + ' per day'] < 0].drop_duplicates(subset=['city']).shape[0]

In [ ]:
# Criamos as colunas `cases per day` e `mortes per day` para sabermos onde isso dá problema

cum_columns = ['cases', 'deaths']
df_city = df_city.merge(
    df_city.groupby('city')[cum_columns].diff(),
    left_index=True, right_index=True, suffixes=['', ' per day']
).fillna({'{} per day'.format(cum_column): df_city[cum_column] for cum_column in cum_columns})

df_city.head()

In [ ]:
solucionar_coluna("cases")
solucionar_coluna("deaths")

In [ ]:
# dropo as colunas antigas incorretas
df_city = df_city.drop(columns=['cases per day', 'deaths per day'])

# recrio as colunas só que com a agregação correta
cum_columns = ['cases', 'deaths']
df_city = df_city.merge(
    df_city.groupby('city')[cum_columns].diff(),
    left_index=True, right_index=True, suffixes=['', ' per day']
).fillna({'{} per day'.format(cum_column): df_city[cum_column] for cum_column in cum_columns})

df_city.head()

In [ ]:
# Verifico se não há nenhuma coluna com problema de acumulação após o processamento
quant_cases = df_city.loc[df_city['cases per day'] < 0].drop_duplicates(subset=['city']).shape[0]
quant_deaths = df_city.loc[df_city['deaths per day'] < 0].drop_duplicates(subset=['city']).shape[0]
print(f"Cidades com o problema na coluna cases após correção: {quant_cases}")
print(f"Cidades com o problema na coluna deaths após correção: {quant_deaths}")

In [ ]:
# Obtenho uma normalização por meio do tamanho da população da cidade
df_city["cases/population"] = df_city["cases"]/df_city["population"]
df_city["deaths/population"] = df_city["deaths"]/df_city["population"]
df_city["cases per day/population"] = df_city["cases per day"]/df_city["population"]
df_city["deaths per day/population"] = df_city["deaths per day"]/df_city["population"]

df_city.rename(columns={'long': 'lon'}, inplace=True)

df_city.head()

## Criação do df_state

In [ ]:
# obtenho o número de casos e mortes por dia por estado
df_state = df_city.groupby(['state', 'date'])[['cases per day', 'deaths per day']].sum()
df_state.reset_index(inplace=True)

In [ ]:
# adiciono a população do estado, somando as populações dos municípios do estado
state_population = df_city.drop_duplicates(subset=['city']).groupby('state')['population'].sum()
population_dict = state_population.to_dict()

def add_population(row):
    return population_dict[row['state']]

df_state['population'] = df_state.apply(add_population, axis=1)
df_state

In [ ]:
# calculo uma média da latitude e longitude dos municípios do estado
state_avg_coord = df_city.drop_duplicates(subset=['city']).groupby('state')[['lat', 'lon']].mean()

# adiciono a lat e long média, além de adicionar a região do estado
df_state = df_state.merge(state_avg_coord, on='state').merge(df_city[['state', 'region']].drop_duplicates(subset=['state']), on='state')

# calculo e adiciono o número de casos e mortes acumulados do estado
df_state['cases'] = df_state.groupby('state')['cases per day'].cumsum(axis=0)
df_state['deaths'] = df_state.groupby('state')['deaths per day'].cumsum(axis=0)

# Obtenho uma normalização por meio do tamanho da população do estado
df_state["cases/population"] = df_state["cases"]/df_state["population"]
df_state["deaths/population"] = df_state["deaths"]/df_state["population"]
df_state["cases per day/population"] = df_state["cases per day"]/df_state["population"]
df_state["deaths per day/population"] = df_state["deaths per day"]/df_state["population"]
df_state

## Criação do gráfico

In [ ]:
df_cities_coordinates = pd.read_csv(data_path + "brazil_cities_coordinates.csv")
df_cities_coordinates.head()

In [ ]:
df_brazil = pd.read_csv(data_path + "brazil_cities_coordinates.csv")
df_brazil.loc[df_brazil["city_name"] == 'Acrelândia']

In [ ]:
brazil_code = 100
with urlopen(f'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-{brazil_code}-mun.json') as response:
    brazil_json = json.loads(response.read())

In [ ]:
df_city.loc[df_city['date'] == '2020-11-23'].describe()
df_city['date'].max()

In [ ]:
df_city.loc[(df_city['date'] == '2020-05-23') & (df_city['city'] == 'São Carlos') & (df_city['state'] == 'SP') & (df_city['region'] == 'Sudeste')]

In [ ]:
df_city.iloc[1444582]

In [ ]:
df_city.loc[(df_city['city'] == 'São Carlos')]

In [ ]:
fig = px.choropleth_mapbox(df_city.iloc[1444582-1: 1444582+1], geojson=brazil_json, featureidkey = 'properties.id', locations='city_code', color='deaths',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=9, center = df_city.iloc[1444582][["lat", "lon"]].to_dict(),
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()